In [ ]:
import os
os.chdir("../")

In [67]:
%pwd

'c:\\Users\\akshay.k\\Desktop\\Chatbot\\CHATBOT_GENERATIVE_AI'

In [68]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [72]:
# Extract data from the pdf loader
def loadPdfFile(data):
    loader = DirectoryLoader(data,
                             glob="*pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [73]:
extracted_data = loadPdfFile(data='data/')

In [74]:
# extracted_data

In [75]:
# split the Data into text chunks
def text_split(extracted_data):
    text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap =20)
    text_chunks = text_spliter.split_documents(extracted_data)
    return text_chunks

In [76]:
text_chunks = text_split(extracted_data)
print("length of text chunks",len(text_chunks))

length of text chunks 335


In [77]:
from langchain.embeddings import HuggingFaceEmbeddings

In [78]:
# download the embedding from hugging face

def download_hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2')
    return embedding


In [79]:
embeddings = download_hugging_face_embedding()

In [80]:
query_result = embeddings.embed_query("hai there")
print("length",len(query_result))

length 384


In [81]:
# query_result

In [87]:
from dotenv import load_dotenv
load_dotenv()


True

In [88]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [89]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = 'chatbot'

pc.create_index(
    name= index_name,
    dimension= 384, # Replace with your model dimensions
    metric= "cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "chatbot",
    "metric": "cosine",
    "host": "chatbot-wd9ztlb.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [90]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [91]:
# embed each chunk and upsert the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name = index_name,
    embedding=embeddings,
)

In [92]:
# load existing index
from langchain_pinecone import PineconeVectorStore
# embed each chunk and upsert the embeddings into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [93]:
docsearch

In [94]:
retriver = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [97]:
retrived_docs = retriver.invoke("is there any automation service you are doing?")

In [ ]:
# retrived_docs

[Document(id='da55c0c9-c120-409a-94d8-611c1afde562', metadata={'author': 'Megha K P', 'creationdate': '2025-05-21T05:54:15+00:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2025-05-21T05:54:15+00:00', 'page': 0.0, 'page_label': '1', 'producer': 'www.ilovepdf.com', 'source': 'data\\Automation Process Mining_Service_11-07-2024.pdf', 'total_pages': 4.0}, page_content='automation strategies that seamlessly integrate into your existing workflows, aligning \nperfectly with your organizational goals. \nOur comprehensive approach combines state-of-the-art technology with strategic foresight, \ndelivering robust RPA solutions that yield measurable results. From initial exploration to full-\nscale implementation, we stand as your trusted partner, guiding you through the intricacies \nof automation with expertise and precision. Embrace a new era of efficiency and growth'),
 Document(id='fa889cc1-aad4-4662-aac4-41d402a45523', metadata={'author': 'Megha K P', 'creationdate': '2025-05-21T05:54:

In [126]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)
from langchain_community.llms import Ollama
llm = Ollama(model="mistral", temperature=0.4)


In [127]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Your name is Dify, and you are an intelligent assistant for Difinity Digital, designed to handle question-answering tasks."
    "If the user asks a casual question (e.g., greetings like 'hi', 'hello', or small talk), respond in a friendly, conversational tone."
    "if the user asks a question about company policies, services, or anything related to Difinity Digital, respond using the retrieved context provided."
    "If you don't know the answer, respond honestly by saying you don't know."
    "Keep all responses to a maximum of three sentences. Be clear, concise, and helpful."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [128]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

In [139]:
response = rag_chain.invoke({"input":"are you there"})
print(response["answer"])

MaxRetryError: HTTPSConnectionPool(host='chatbot-wd9ztlb.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001E933A7E050>: Failed to resolve 'chatbot-wd9ztlb.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))